# Metadata Analysis

In [1]:
import pandas as pd
import numpy as np
import re
%load_ext autoreload
%autoreload 2
from cord.cord19 import ResearchPapers
from pathlib import Path, PurePath
from IPython.display import display

pd.options.display.max_colwidth=200
pd.options.display.max_rows = 4000

In [2]:
data_path = Path('data') / 'CORD-19-research-challenge'
metadata_path = PurePath(data_path) / 'metadata.csv'

## Load Metadata

In [3]:
from cord.core import describe_dataframe
metadata = ResearchPapers.load_metadata()
describe_dataframe(metadata)

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata
Applying tags to metadata


,non-null,null,unique,duplicate,most common
cord_uid,128492,0,128162,330,f8hgcngj
sha,55751,72741,55748,3,0ed3c6a5559cd73307184f51fc53ccc76da559bc
source,128492,0,33,128459,Medline
title,128492,0,118159,10333,
doi,128492,0,100162,28330,
pmcid,60771,67721,60771,0,PMC7184393
pubmed_id,99124,29368,92197,6927,15473650
license,128492,0,17,128475,unk
abstract,128492,0,118732,9760,
published,128477,15,7100,121377,2020-01-01 00:00:00


In [4]:
papers = ResearchPapers.load()

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata
Applying tags to metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 98.0 seconds


In [5]:
papers

In [17]:
metadata_nonnulls = metadata.dropna(subset=['url'])
metadata_multi = metadata_nonnulls[metadata_nonnulls.url.str.contains(';')]

In [25]:
metadata_multi[metadata_multi.url.str.contains('https://api.elsevier')]

,cord_uid,sha,source,title,doi,pmcid,pubmed_id,license,abstract,published,...,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,when,covid_related,virus,coronavirus,sars
57460,07redqbb,NaN,Elsevier,,10.1016/b978-0-12-812343-0.09995-1,NaN,NaN,els-covid,,2017-12-31,...,NaN,NaN,NaN,https://api.elsevier.com/content/article/pii/B9780128123430099951; https://www.sciencedirect.com/science/article/pii/B9780128123430099951,214757615.0,2 years ago,False,False,False,False
57475,dthdszdn,75cf04320213a3ed0e444333d095f803065bd0ee,Elsevier,Cardiovascular implications of the COVID-19 pandemic: a global perspective,10.1016/j.cjca.2020.05.018,NaN,NaN,els-covid,"The Coronavirus disease 2019 (COVID-19), caused by severe acute respiratory syndrome coronavirus-2 (SARS-CoV-2), represents the pandemic of the century, with approximately 3.5 million cases and 2...",2020-05-16,...,NaN,document_parses/pdf_json/75cf04320213a3ed0e444333d095f803065bd0ee.json,NaN,https://api.elsevier.com/content/article/pii/S0828282X20304645; https://www.sciencedirect.com/science/article/pii/S0828282X20304645?v=s5,218657226.0,2 weeks ago,True,True,True,False
57478,3qmag7ny,NaN,Elsevier,"The Political Economy of the SARS Epidemic: The Impact on Human Resources in East Asia, Grace O.M. Lee, Malcolm Warner, Routledge London and New York, 2008, xxii+168 pp. $19.95, ISBN: 0-415-39498-8.",10.1016/j.asieco.2008.09.008,NaN,NaN,els-covid,"The Political Economy of the SARS Epidemic: The Impact on Human Resources in East Asia, Grace O.M. Lee, Malcolm Warner, Routledge London and New York, 2008, xxii+168 pp. $19.95, ISBN: 0-415-39498-8.",2009-01-31,...,NaN,NaN,NaN,https://api.elsevier.com/content/article/pii/S1049007808000900; https://www.sciencedirect.com/science/article/pii/S1049007808000900,152480814.0,11 years ago,False,False,False,True
57480,sa2c3z14,62cc1be8f41f61f0b55f07df753b74e2f4c4b4ce,Elsevier,Manejo de pacientes de Ortopedia y Traumatología en el contexto de la contingencia por covid-19: revisión de conceptos actuales,10.1016/j.rccot.2020.05.001,NaN,NaN,els-covid,"Resumen El objetivo del presente estudio es presentar una revisión de la literatura disponible, que permita abordar de forma ordenada la evidencia actual con respecto a la organización de un servi...",2020-05-16,...,NaN,document_parses/pdf_json/62cc1be8f41f61f0b55f07df753b74e2f4c4b4ce.json,NaN,https://api.elsevier.com/content/article/pii/S0120884520300468; https://www.sciencedirect.com/science/article/pii/S0120884520300468?v=s5,218656924.0,2 weeks ago,True,False,False,False
57485,5kohk0c3,46c645c9ad604d7cb3740e444618b75e32012d66,Elsevier,"Impact of urbanization on pollution-related agricultural input intensity in Hubei, China",10.1016/j.ecolind.2015.11.002,NaN,NaN,els-covid,"Agricultural input intensity increases significantly during the rapid urbanization in China, which has contributed to the increasingly serious non-point pollution. Using the vector autoregression...",2016-03-31,...,NaN,document_parses/pdf_json/46c645c9ad604d7cb3740e444618b75e32012d66.json,NaN,https://api.elsevier.com/content/article/pii/S1470160X15006330; https://www.sciencedirect.com/science/article/pii/S1470160X15006330,86132905.0,4 years ago,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128482,ks6tcyka,1f0097fb57b078c0e81d219f65f218d571314be2,Elsevier; Medline; PMC,APOBEC3G directly binds Hepatitis B virus core protein in cell and cell free systems,10.1016/j.virusres.2010.05.009,PMC7173111,20510315,els-covid,APOBEC3G (A3G) is an intrinsic antiretroviral factor which can inhibit Hepatitis B virus (HBV) replication. This antiviral activity mainly depends on A3G incorporation into viral particles. Howev...,2010-08-31,...,NaN,document_parses/pdf_json/1f0097fb57b078c0e81d219f65f218d571314be2.json,document_parses/pmc_json/PMC7173111.xml.json,https://api.elsevier.com/content/article/pii/S0168170210001760; https://www.sciencedirect.com/science/article/pii/S0168170210001760,22731338.0,1

In [39]:
papers[128491].url

'https://www.sciencedirect.com/science/article/pii/S0261517709001745'